In [392]:
# Import base dependencies
import pandas as pd
import requests
import time
from datetime import datetime
import math
from config import av_api_key as api_key
# import matplotlib.pyplot as plt
# from sklearn.linear_model import LinearRegression

# Import ML dependencies
# import numpy as np
# import tensorflow as tf
# from tensorflow import keras
# from keras.models import Sequential
# from keras.layers import SimpleRNN, Flatten, TimeDistributed, LSTM

In [393]:
# Import raw data from platform export
# IRA funds
ira_funds = [{'symbol':"LRN"}, 
             {'symbol':"UNFI", 'basis':22.360}, 
             {'symbol':"MFC"}, 
             {'symbol':"EAT", 'basis':112.486}, 
             {'symbol':"EZPW", 'basis':15.460}, 
             {'symbol':"ARQT", 'basis':15.932}, 
             {'symbol':"WFC", 'basis':65.815}, 
             {'symbol':"ITRN", 'basis':35.710}, 
             {'symbol':"CRDO", 'basis':22.622}, 
             {'symbol':"PYPL", 'basis':69.910}, 
             {'symbol':"ALL"}, 
             {'symbol':"LC", 'basis':9.900}, 
             {'symbol':"QTWO"}, 
             {'symbol':"CLS", 'basis':54.610}, 
             {'symbol':"CCL", 'basis':16.147}, 
             {'symbol':"AGX", 'basis':41.148}, 
             {'symbol':"POWL", 'basis':187.000}, 
             {'symbol':"PPC", 'basis':45.054}, 
             {'symbol':"SYF"}, 
             {'symbol':"ATGE"}, 
             {'symbol':"BRK-B"}, 
             {'symbol':"SFM"}, 
             {'symbol':"SKYW", 'basis':78.359}, 
             {'symbol':"BLBD"}, 
             {'symbol':"GM"}, 
             {'symbol':"RCL"}, 
             {'symbol':"OKTA"}, 
             {'symbol':"TWLO", 'basis':98.403}, 
             {'symbol':"APP", 'basis':248.047}, 
             {'symbol':"TMUS"}, 
             {'symbol':"STRL"}, 
             {'symbol':"GRBK"}, 
             {'symbol':"UBER", 'basis':50.777}, 
             {'symbol':"CAAP", 'basis':21.57}
             ]

# Brokerage
brokerage_funds = [{'symbol':"AGX", 'basis':86.654}, 
                   {'symbol':"APP", 'basis':225.605}, 
                   {'symbol':"ARQT", 'basis':15.996}, 
                   {'symbol':"ATGE", 'basis':86.385}, 
                   {'symbol':"CCL", 'basis':20.112}, 
                   {'symbol':"CLS", 'basis':74.924}, 
                   {'symbol':"COMM"}, 
                   {'symbol':"CRDO", 'basis':38.629}, 
                   {'symbol':"EAT", 'basis':135.74}, 
                   {'symbol':"EZPW", 'basis':18.176}, 
                   {'symbol':"FBTC", 'basis':75.244, 'is_etf': True}, 
                   {'symbol':"NVDA", 'basis':95.19}, 
                   {'symbol':"NVTS"}, 
                   {'symbol':"OKTA", 'basis':106.18}, 
                   {'symbol':"SKYW", 'basis':84.188}, 
                   {'symbol':"STRL"}, 
                   {'symbol':"TSM"}, 
                   {'symbol':"TWLO", 'basis':92.001}, 
                   {'symbol':"UBER"}, 
                   {'symbol':"UNFI", 'basis':23.000}, 
                   {'symbol':"WFC", 'basis':68.586}
                   ]

In [394]:
def get_history(symbol, api_key, days=252):  # ~1 year default
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&apikey={api_key}&outputsize=full&entitlement=delayed"
    try:
        response = requests.get(url).json()
        if "Time Series (Daily)" not in response:
            error_msg = response.get('Note', response.get('Information', 'Unknown error'))
            print(f"Error fetching price data for {symbol}: {error_msg}")
            print(f"Response keys: {list(response.keys())}")
            return None
        
        time_series = response["Time Series (Daily)"]
        df = pd.DataFrame.from_dict(time_series, orient="index", dtype=float)
        
        # Debug: Print available columns
        # print(f"Columns for {symbol}: {list(df.columns)}")
        
        # Rename columns dynamically
        column_map = {
            col: name for col, name in [
                ("1. open", "Open"), ("2. high", "High"), ("3. low", "Low"),
                ("4. close", "Close"), ("5. volume", "Volume"), ("6. volume", "Volume"),
                ("7. adjusted close", "Adjusted Close"), ("8. dividend amount", "Dividend")
            ] if col in df.columns
        }
        if "5. volume" not in df.columns and "6. volume" not in df.columns:
            print(f"No volume data for {symbol}")
            return None
        
        df = df.rename(columns=column_map)
        df.index = pd.to_datetime(df.index)
        df = df.sort_index().tail(days)
        return df
    except Exception as e:
        print(f"Exception fetching price data for {symbol}: {str(e)}")
        return None

def get_fundamentals(symbol, api_key, current_price):
    url = f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={api_key}&entitlement=delayed"
    try:
        response = requests.get(url).json()
        if not response or "Symbol" not in response:
            error_msg = response.get('Note', response.get('Information', 'No data'))
            print(f"Error fetching fundamentals for {symbol}: {error_msg}")
            print(f"Full response: {response}")
            return None
        
        def safe_float(value, default):
            if value in [None, 'None', '']:
                return default
            try:
                return float(value)
            except (ValueError, TypeError):
                return default
        
        pe_ratio = safe_float(response.get('PERatio'), float('inf'))
        pb_ratio = safe_float(response.get('PriceToBookRatio'), float('inf'))
        
        # Calculate EPS and Book Value
        eps = current_price / pe_ratio if pe_ratio != float('inf') and pe_ratio != 0 else 0
        book_value = current_price / pb_ratio if pb_ratio != float('inf') and pb_ratio != 0 else 0
        
        fundamentals = {
            'pe_ratio': pe_ratio,
            'pb_ratio': pb_ratio,
            'dividend_yield': safe_float(response.get('DividendYield'), 0),
            'debt_to_equity': safe_float(response.get('DebtToEquityRatio'), float('inf')),
            'eps': eps,
            'book_value': book_value
        }
        
        return fundamentals
    except Exception as e:
        print(f"Exception fetching fundamentals for {symbol}: {str(e)}")
        return None

def calculate_vwap(df, days=126):  # ~2 quarters
    if 'Volume' not in df.columns:
        print("Missing Volume column in DataFrame")
        return None
    
    vwap_analysis = df[-days:].copy()
    vwap_analysis['Cumulative_LTPV'] = (vwap_analysis['Low'] * vwap_analysis['Volume']).cumsum()
    vwap_analysis['Cumulative_HTPV'] = (vwap_analysis['High'] * vwap_analysis['Volume']).cumsum()
    vwap_analysis['Cumulative_Volume'] = vwap_analysis['Volume'].cumsum()
    vwap_analysis['Entry'] = round(vwap_analysis['Cumulative_LTPV'] / vwap_analysis['Cumulative_Volume'], 2)
    vwap_analysis['Exit'] = round(vwap_analysis['Cumulative_HTPV'] / vwap_analysis['Cumulative_Volume'], 2)
    return vwap_analysis[-1:].copy()

def build_analysis_table(ticker_symbols, api_key, margin_of_safety=0.9, vwap_days=126, graham_margin=0.95):
    portfolio = []
    
    for ticker in ticker_symbols:
        symbol = ticker['symbol']
        is_etf = ticker.get('is_etf', False)
        
        # Get price data
        raw_data = get_history(symbol, api_key)
        if raw_data is None:
            portfolio.append([symbol, ticker.get('basis', 0), None, None, None, None, None, None, None, "Error"])
            continue
        
        # Get fundamentals (skip for ETFs)
        current_price = raw_data['Close'].iloc[-1]  # Use Close for fundamental calcs
        fundamentals = None if is_etf else get_fundamentals(symbol, api_key, current_price)
        if not is_etf and fundamentals is None:
            portfolio.append([symbol, ticker.get('basis', 0), None, None, None, None, None, None, None, "Error"])
            continue
        
        # Calculate VWAP
        vwap_data = calculate_vwap(raw_data, days=vwap_days)
        if vwap_data is None:
            portfolio.append([symbol, ticker.get('basis', 0), None, None, None, None, None, None, None, "Error"])
            continue
        
        # Extract data
        basis = ticker.get('basis', 0)
        market_price = round(raw_data['Close'].iloc[-1], 2)  # Use Low for buys
        entry_price = round(vwap_data['Entry'].iloc[0], 2)
        exit_price = round(vwap_data['Exit'].iloc[0], 2)
        buy_threshold = round(entry_price * margin_of_safety, 2)  # 10% margin
        
        # Graham buy threshold (for stocks only)
        graham_buy_threshold = None
        if not is_etf:
            if fundamentals['eps'] > 0 and fundamentals['book_value'] > 0:
                # Calculate desired price where P/E × P/B = 36
                desired_price = math.sqrt(38 * fundamentals['eps'] * fundamentals['book_value'])
                graham_buy_threshold = round(desired_price * graham_margin, 2)  # 5% margin
            else:
                graham_buy_threshold = buy_threshold  # Default to VWAP threshold
        
        # Volume filter: 20% of 21-day average
        avg_volume = raw_data['Volume'][-21:].mean()
        today_volume = raw_data['Volume'].iloc[-1]
        volume_ok = today_volume >= avg_volume * 0.2
        
        # Graham's fundamental checks (for stocks only)
        graham_ok = True
        if not is_etf:
            graham_ok = (
                (fundamentals['pe_ratio'] < 19 and fundamentals['pb_ratio'] < 2.0) or
                (fundamentals['pe_ratio'] * fundamentals['pb_ratio'] < 38 and 
                 fundamentals['pe_ratio'] < 100 and fundamentals['pb_ratio'] < 10)
                 ) and fundamentals['dividend_yield'] >= 0 and fundamentals['debt_to_equity'] < 2
        
        # Decision logic
        decision = "Hold"
        if market_price <= min(buy_threshold, graham_buy_threshold or float('inf')) and volume_ok and graham_ok:
            decision = "Buy"
        elif market_price >= exit_price and volume_ok:
            decision = "Sell"
        
        # Prepare fundamentals for output
        pe_ratio = None if is_etf else fundamentals['pe_ratio']
        pb_ratio = None if is_etf else fundamentals['pb_ratio']
        dividend_yield = None if is_etf else fundamentals['dividend_yield']
        
        portfolio.append([
            symbol, basis, market_price, 
            buy_threshold, graham_buy_threshold, exit_price,
            pe_ratio, pb_ratio, dividend_yield, decision
        ])
        
        # Minimal delay for server stability (75 calls/minute = ~0.8 seconds/call)
        time.sleep(0.1)
    
    return portfolio

In [395]:
b_portfolio = build_analysis_table(brokerage_funds, api_key, margin_of_safety=1, vwap_days=63)
brokerage_df = pd.DataFrame(b_portfolio, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

In [396]:
brokerage_df[brokerage_df['basis']>0]

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,AGX,86.654,218.17,163.45,81.03,174.73,30.39,8.180,0.0069,Sell
1,APP,225.605,364.49,294.20,17.98,319.58,65.79,214.340,0.0000,Sell
2,ARQT,15.996,13.43,13.93,13.93,15.14,inf,11.280,0.0000,Hold
3,ATGE,86.385,121.84,111.73,89.90,117.03,20.62,3.055,0.0000,Sell
4,CCL,20.112,22.41,19.84,18.98,20.81,14.46,3.306,0.0000,Sell
5,CLS,74.924,125.33,90.98,39.07,97.41,36.54,9.660,0.0000,Sell
7,CRDO,38.629,73.49,51.56,6.32,55.82,253.41,18.310,0.0000,Sell
8,EAT,135.740,174.57,144.95,37.72,153.28,24.28,30.260,0.0000,Sell
9,EZPW,18.176,13.17,14.81,23.64,15.39,11.26,0.945,0.0000,Hold
10,FBTC,75.244,91.79,80.34,NaN,82.70,NaN,NaN,NaN,Sell


In [397]:
brokerage_df[brokerage_df['basis']==0]

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
6,COMM,0.0,5.81,4.73,4.73,5.12,inf,30.360,0.0000,Sell
12,NVTS,0.0,7.32,5.12,5.12,6.40,inf,4.111,0.0000,Sell
15,STRL,0.0,202.99,143.26,88.15,151.51,23.74,7.660,0.0000,Sell
16,TSM,0.0,211.10,168.62,91.94,175.04,25.25,7.160,0.0156,Sell
18,UBER,0.0,83.93,78.58,44.91,81.78,14.70,8.150,0.0000,Sell


In [398]:
r_portfolio = build_analysis_table(ira_funds, api_key, margin_of_safety=.95, vwap_days=63)
retirement_df = pd.DataFrame(r_portfolio, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

In [399]:
retirement_df[retirement_df['basis']>0]

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
1,UNFI,22.360,21.30,24.22,24.22,26.96,inf,0.804,0.0000,Hold
3,EAT,112.486,174.57,137.70,37.72,153.28,24.28,30.260,0.0000,Sell
4,EZPW,15.460,13.17,14.07,23.64,15.39,11.26,0.945,0.0000,Hold
5,ARQT,15.932,13.43,13.23,13.23,15.14,inf,11.280,0.0000,Hold
6,WFC,65.815,72.36,65.47,97.60,71.26,13.01,1.449,0.0221,Sell
7,ITRN,35.710,36.23,33.47,30.17,36.54,13.08,3.780,0.0552,Hold
8,CRDO,22.622,73.49,48.98,6.32,55.82,253.41,18.310,0.0000,Sell
9,PYPL,69.910,70.83,62.50,56.37,68.08,15.92,3.401,0.0000,Sell
11,LC,9.900,10.46,9.46,13.43,10.47,23.77,0.875,0.0000,Hold
13,CLS,54.610,125.33,86.43,39.07,97.41,36.54,9.660,0.0000,Sell


In [400]:
retirement_df[retirement_df['basis']==0]

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,LRN,0.0,143.25,131.45,84.34,144.81,22.49,4.399,0.0000,Hold
2,MFC,0.0,31.18,28.47,35.76,30.87,16.16,1.613,0.0407,Sell
10,ALL,0.0,198.92,187.56,194.96,203.07,13.59,2.627,0.0201,Hold
12,QTWO,0.0,85.68,75.84,75.84,83.09,inf,9.790,0.0000,Sell
18,SYF,0.0,59.84,49.13,100.49,54.15,8.20,1.483,0.0201,Sell
19,ATGE,0.0,121.84,106.14,89.90,117.03,20.62,3.055,0.0000,Sell
20,BRK-B,0.0,487.54,481.55,624.90,519.60,12.99,1.607,0.0000,Hold
21,SFM,0.0,157.92,148.80,44.76,163.37,35.57,12.000,0.0000,Hold
23,BLBD,0.0,40.80,33.60,25.82,36.93,12.67,6.760,0.0000,Sell
24,GM,0.0,48.65,44.10,128.23,48.08,6.79,0.727,0.0123,Sell


In [401]:
# Research new investments
watch_list = [{'symbol':'BSAC'}, # Chile - Latin America - Financials
              {'symbol':'KARO'}, # Singapore - Technology - SaaS
              {'symbol':'QFIN'}, # China - FinTech
              {'symbol':'TSM'}, # Taiwan - Semiconductors
              {'symbol':'CCU'}, # Chile - Consumer Staples - beverage distributor
              {'symbol':'ABEV'}, # Brazil - Financials - but distributes beers?
              {'symbol':'TIMB'}, # Brazil - Communications - Telecom (mobile networking)
              {'symbol':'ASR'}, # Mexico - Industrials - Airport operations in South America
              {'symbol':'RYAAY'}, # Ireland - Industrials - budget airline
              {'symbol':'AMX'} # Mexico - Communications - mobile network
             ] 
watch_table = build_analysis_table(watch_list, api_key, margin_of_safety=1, vwap_days=63)
internationals = build_analysis_table(watch_list, api_key, margin_of_safety=0.95, vwap_days=126)

# Create watch list data frame
watch_list_df = pd.DataFrame(watch_table, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

internationals_df = pd.DataFrame(internationals, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

In [402]:
watch_list_df

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,BSAC,0,24.76,23.43,28.02,24.04,10.81,2.478,0.0545,Sell
1,KARO,0,48.24,46.98,17.94,50.15,28.71,8.640,0.0224,Hold
2,QFIN,0,41.69,40.85,72.80,43.18,6.43,1.749,0.0312,Hold
3,TSM,0,211.10,168.62,91.94,175.04,25.25,7.160,0.0156,Sell
4,CCU,0,12.99,14.44,17.11,14.89,13.53,1.461,0.0351,Hold
5,ABEV,0,2.44,2.38,2.45,2.45,15.25,2.233,0.0626,Hold
6,TIMB,0,18.59,16.20,20.02,16.58,14.52,2.036,0.0830,Sell
7,ASR,0,314.03,303.06,279.61,312.40,13.16,3.287,0.0783,Sell
8,RYAAY,0,55.22,48.92,41.86,50.17,16.48,3.622,0.0169,Sell
9,AMX,0,17.41,15.85,10.55,16.26,35.53,2.630,0.0291,Sell


In [403]:
internationals_df

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,BSAC,0,24.76,21.35,28.02,23.01,10.81,2.478,0.0545,Sell
1,KARO,0,48.24,43.62,17.94,48.70,28.71,8.640,0.0224,Hold
2,QFIN,0,41.69,38.28,72.80,42.41,6.43,1.749,0.0312,Hold
3,TSM,0,211.10,172.88,91.94,188.92,25.25,7.160,0.0156,Sell
4,CCU,0,12.99,12.85,17.11,13.94,13.53,1.461,0.0351,Hold
5,ABEV,0,2.44,2.01,2.45,2.18,15.25,2.233,0.0626,Sell
6,TIMB,0,18.59,13.78,20.02,14.83,14.52,2.036,0.0830,Sell
7,ASR,0,314.03,272.71,279.61,296.19,13.16,3.287,0.0783,Sell
8,RYAAY,0,55.22,44.75,41.86,48.34,16.48,3.622,0.0169,Sell
9,AMX,0,17.41,14.49,10.55,15.66,35.53,2.630,0.0291,Sell


In [404]:
# Research new investments
mag7_test = [{'symbol':'MSFT'}, 
             {'symbol':'AAPL'}, 
             {'symbol':'NVDA'}, 
             {'symbol':'AMZN'}, 
             {'symbol':'GOOG'}, 
             {'symbol':'GOOGL'}, 
             {'symbol':'META'}, 
             {'symbol':'BRK-B'}, 
             {'symbol':'TSLA'}
             ] 
mag7_test_table = build_analysis_table(mag7_test, api_key, vwap_days=63)

# Create watch list data frame
mag7_df = pd.DataFrame(mag7_test_table, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

In [405]:
mag7_test_table

[['MSFT', 0, 474.96, 365.36, 138.51, 416.24, 36.76, 10.97, 0.007, 'Sell'],
 ['AAPL', 0, 196.45, 178.43, 31.16, 206.23, 30.6, 44.54, 0.0053, 'Hold'],
 ['NVDA', 0, 141.97, 103.1, 19.09, 120.12, 45.95, 41.3, 0.0003, 'Sell'],
 ['AMZN', 0, 212.1, 169.48, 77.84, 195.59, 34.6, 7.36, 0, 'Sell'],
 ['GOOG', 0, 175.88, 144.46, 93.47, 166.19, 19.65, 6.18, 0.0048, 'Sell'],
 ['GOOGL', 0, 174.67, 142.79, 93.51, 164.38, 19.49, 6.14, 0.0048, 'Sell'],
 ['META', 0, 682.87, 515.88, 252.35, 596.57, 26.66, 9.42, 0.0031, 'Sell'],
 ['BRK-B', 0, 487.54, 456.21, 624.9, 519.6, 12.99, 1.607, 0, 'Hold'],
 ['TSLA', 0, 325.31, 248.19, 37.4, 295.01, 184.84, 14.04, 0, 'Sell']]

In [406]:
mag7_df

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,MSFT,0,474.96,365.36,138.51,416.24,36.76,10.970,0.0070,Sell
1,AAPL,0,196.45,178.43,31.16,206.23,30.60,44.540,0.0053,Hold
2,NVDA,0,141.97,103.10,19.09,120.12,45.95,41.300,0.0003,Sell
3,AMZN,0,212.10,169.48,77.84,195.59,34.60,7.360,0.0000,Sell
4,GOOG,0,175.88,144.46,93.47,166.19,19.65,6.180,0.0048,Sell
5,GOOGL,0,174.67,142.79,93.51,164.38,19.49,6.140,0.0048,Sell
6,META,0,682.87,515.88,252.35,596.57,26.66,9.420,0.0031,Sell
7,BRK-B,0,487.54,456.21,624.90,519.60,12.99,1.607,0.0000,Hold
8,TSLA,0,325.31,248.19,37.40,295.01,184.84,14.040,0.0000,Sell


In [407]:
# Machine Learning

In [408]:
# Data processing and clearning
# Must be in numpy array or tf.Dataset object format

In [409]:
# Feature selection and normalization

In [410]:
# Build model

In [411]:
# Train model

In [412]:
# Evaluate model

In [413]:
# Refine model through hyperparameter tuning